In [19]:
import pandas as pd
import os
import numpy as np
from itertools import product

#os - operating system dependent. to read/write files
#pd - pandas for data analysis/modeling

data_path = 'data/'

#get DataFrames
#Reformated sales into one dataframe in 3.1_CreateMonthlySales
trainingMonthSet    = pd.read_pickle(os.path.join(data_path, 'trainingMonthSet.pickle'))

In [15]:
trainingMonthSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609124 entries, 0 to 2935847
Data columns (total 6 columns):
shop_id             1609124 non-null int8
item_id             1609124 non-null int16
date_block_num      1609124 non-null int8
item_category_id    1609124 non-null int8
item_price          1609124 non-null float64
item_cnt_month      1609124 non-null int16
dtypes: float64(1), int16(2), int8(3)
memory usage: 35.3 MB


<h3>Creating zero sales data</h3>
<p>Creating a dataframe with all the possible shop and item combinations for that particular month, since the sales file only includes items which has any sales or returns it doesnt include data which did not had any sale. Thefore we create a new dataset with those items assuming every shop contains those items</p>

In [11]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in trainingMonthSet['date_block_num'].unique():
    cur_shops = trainingMonthSet.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = trainingMonthSet.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [12]:
grid.shape

(10913850, 3)

In [20]:
all_data = pd.merge(grid, trainingMonthSet , how='left', on=index_cols).fillna(0)
all_data.head()

,shop_id,item_id,date_block_num,date,item_category_id,item_price,item_cnt_month
0,59,22154,0,2013-01-01 00:00:00,37.0,999.0,1.0
1,59,2552,0,0,0.0,0.0,0.0
2,59,2554,0,0,0.0,0.0,0.0
3,59,2555,0,0,0.0,0.0,0.0
4,59,2564,0,0,0.0,0.0,0.0


In [24]:
import datetime
for i in range(0,34):
    all_data.loc[all_data.date_block_num==i,'date']=datetime.date(2013+(int)(i/12), 1+i%12, 1)

In [ ]:
for i in range(0,34):
    for j in range(0,)
    all_data.loc[all_data.date_block_num==i,'date'].loc[all_data.item_id]=datetime.date(2013+(int)(i/12), 1+i%12, 1)

In [26]:
all_data.tail()

,shop_id,item_id,date_block_num,date,item_category_id,item_price,item_cnt_month
10913845,21,7635,33,2015-10-01,0.0,0.0,0.0
10913846,21,7638,33,2015-10-01,0.0,0.0,0.0
10913847,21,7640,33,2015-10-01,0.0,0.0,0.0
10913848,21,7632,33,2015-10-01,0.0,0.0,0.0
10913849,21,7440,33,2015-10-01,0.0,0.0,0.0


In [17]:
all_data.shape

(10913850, 6)

<h3>Get shop wise, item wise, item-category wise data</h3>

In [18]:
# Groupby data to get shop-month aggregates to get rid of duplicates
gbShopSum = sales.groupby(['date_block_num','shop_id'],as_index=False).agg({'item_cnt_day':'sum'}).astype(np.int32)
gbShopMean = sales.groupby(['date_block_num','shop_id'],as_index=False).agg({'item_cnt_day':np.mean})
gbShop = pd.merge(gbShopSum, gbShopMean, how='left', on=['date_block_num','shop_id']).fillna(0)
gbShop.columns = ['date_block_num','shop_id','shop_block_target_sum','shop_block_target_mean']
gbShop.head()

,date_block_num,shop_id,shop_block_target_sum,shop_block_target_mean
0,0,0,5578,1.163781
1,0,1,2947,1.093912
2,0,2,1146,1.070028
3,0,3,767,1.024032
4,0,4,2114,1.116156


In [9]:
gbShop.head()

,date_block_num,shop_id,shop_block_target_sum,shop_block_target_mean
0,0,0,5578,1.163781
1,0,1,2947,1.093912
2,0,2,1146,1.070028
3,0,3,767,1.024032
4,0,4,2114,1.116156


In [10]:
# Groupby data to get item-month aggregates to get rid of duplicates
gbItemSum = sales.groupby(['date_block_num','item_id'],as_index=False).agg({'item_cnt_day':'sum'})
gbItemMean = sales.groupby(['date_block_num','item_id'],as_index=False).agg({'item_cnt_day':np.mean})
gbItem = pd.merge(gbItemSum, gbItemMean, how='left', on=['date_block_num','item_id']).fillna(0)
gbItem.columns = ['date_block_num','item_id','item_block_target_sum','item_block_target_mean']
gbItem.head()

,date_block_num,item_id,item_block_target_sum,item_block_target_mean
0,0,19,1,1.000000
1,0,27,7,1.000000
2,0,28,8,1.000000
3,0,29,4,0.666667
4,0,32,299,1.328889


In [11]:
salesWithCat=pd.merge(sales,items[['item_id','item_category_id']],on=['item_id'],how='left')
salesWithCat.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-01-02,0,59,22154,999.00,1,37
1,2013-01-03,0,25,2552,899.00,1,58
2,2013-01-05,0,25,2552,899.00,-1,58
3,2013-01-06,0,25,2554,1709.05,1,58
4,2013-01-15,0,25,2555,1099.00,1,56


In [12]:
# Groupby data to get category-month aggregates to get rid of duplicates

gbItemCatSum = salesWithCat.groupby(['date_block_num','item_category_id'],as_index=False).agg({'item_cnt_day':'sum'})
gbItemCatMean = salesWithCat.groupby(['date_block_num','item_category_id'],as_index=False).agg({'item_cnt_day':np.mean})
gbItemCat = pd.merge(gbItemCatSum, gbItemCatMean, how='left', on=['date_block_num','item_category_id']).fillna(0)
gbItemCat.columns = ['date_block_num','item_category_id','item_cat_block_target_sum','item_cat_block_target_mean']
gbItemCat.head()

,date_block_num,item_category_id,item_cat_block_target_sum,item_cat_block_target_mean
0,0,0,1.0,1.000000
1,0,1,1.0,1.000000
2,0,2,1390.0,1.100554
3,0,3,440.0,1.317365
4,0,4,251.0,1.054622


<h3>Add other features</h3>

In [13]:
# merge with category id
all_data =pd.merge(all_data,items[['item_id','item_category_id']],on=['item_id'],how='left')
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id
0,59,22154,0,1.0,37
1,59,2552,0,0.0,58
2,59,2554,0,0.0,58
3,59,2555,0,0.0,56
4,59,2564,0,0.0,59


In [14]:
all_data = pd.merge(all_data, gbItemCat, how='left', on=['date_block_num','item_category_id']).fillna(0)
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_block_target_sum,item_cat_block_target_mean
0,59,22154,0,1.0,37,6094.0,1.053414
1,59,2552,0,0.0,58,287.0,0.969595
2,59,2554,0,0.0,58,287.0,0.969595
3,59,2555,0,0.0,56,268.0,1.007519
4,59,2564,0,0.0,59,701.0,0.990113


In [15]:
all_data = pd.merge(all_data, gbItem, how='left', on=['date_block_num','item_id']).fillna(0)
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_block_target_sum,item_cat_block_target_mean,item_block_target_sum,item_block_target_mean
0,59,22154,0,1.0,37,6094.0,1.053414,18,1.0
1,59,2552,0,0.0,58,287.0,0.969595,0,0.0
2,59,2554,0,0.0,58,287.0,0.969595,1,1.0
3,59,2555,0,0.0,56,268.0,1.007519,2,1.0
4,59,2564,0,0.0,59,701.0,0.990113,5,1.0


In [16]:
all_data = pd.merge(all_data, gbShop, how='left', on=['date_block_num','shop_id']).fillna(0)
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_block_target_sum,item_cat_block_target_mean,item_block_target_sum,item_block_target_mean,shop_block_target_sum,shop_block_target_mean
0,59,22154,0,1.0,37,6094.0,1.053414,18,1.0,2017,1.092041
1,59,2552,0,0.0,58,287.0,0.969595,0,0.0,2017,1.092041
2,59,2554,0,0.0,58,287.0,0.969595,1,1.0,2017,1.092041
3,59,2555,0,0.0,56,268.0,1.007519,2,1.0,2017,1.092041
4,59,2564,0,0.0,59,701.0,0.990113,5,1.0,2017,1.092041


In [17]:
all_data.shape

(10913850, 11)

In [18]:
all_data.to_pickle(data_path+'new_sales.pickle',index=False)


In [19]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913850 entries, 0 to 10913849
Data columns (total 11 columns):
shop_id                       int32
item_id                       int32
date_block_num                int32
item_cnt_month                float64
item_category_id              int64
item_cat_block_target_sum     float64
item_cat_block_target_mean    float64
item_block_target_sum         int16
item_block_target_mean        float64
shop_block_target_sum         int32
shop_block_target_mean        float64
dtypes: float64(5), int16(1), int32(4), int64(1)
memory usage: 770.2 MB
